1.Необходимые библиотеки Python: tkinter, PIL, pandas, astropy, numpy

Изменение кода: измените значение files_name на директорию, где находится папка 'annotation system', например: './annotation system/'

Перед запуском этого кода необходимо загрузить изображения на локальный компьютер, путь сохранения должен быть: files_name + 'fts images'. Для загрузки можно запустить код 'downlowaded.ipynb'

Все данные аннотаций будут сохранены в файле 'annotations.csv'. При повторной аннотации одного и того же изображения будет сохранена последняя версия.

2.Метод использования

Запустите программу, интерфейс отобразит первое изображение для аннотации

Выберите категорию для аннотации (например, "filter")

Нажмите левую кнопку мыши и перетащите, чтобы создать прямоугольник, выделяющий соответствующий элемент

Повторите вышеуказанные шаги для всех шести категорий (при маркировке других категорий предыдущий прямоугольник исчезнет, но информация уже записана)

Нажмите кнопку "Save", чтобы сохранить аннотации текущего изображения

Система автоматически загрузит следующее изображение для аннотации

3.Описание интерфейса

Область отображения изображения: в центре главного окна, показывает текущее изображение FITS

Кнопки категорий: шесть кнопок внизу окна для выбора текущей категории аннотации

Строка состояния: показывает текущую выбранную категорию аннотации

Имя изображения: отображает имя файла текущего аннотируемого изображения

Кнопки операций:

Save: сохранить результаты аннотации текущего изображения

Previous Image: загрузить предыдущее изображение

Next Image: загрузить следующее изображение

4.Замечания

Если текстовая область для определенной категории неясна или прерывиста, не аннотируйте эту категорию для данного изображения

Результаты аннотаций автоматически сохраняются в файл annotations.csv

Программа запомнит позицию последней аннотации и при следующем запуске автоматически продолжит с этого места

Прямоугольник аннотации должен как можно точнее охватывать целевой элемент

Данные аннотаций хранятся в виде координатных кортежей в формате (x1, y1, x2, y2)

In [ ]:
import tkinter as tk 
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pandas as pd
import os
import glob
import gzip
import numpy as np
from astropy.io import fits
import shutil

root = tk.Tk()
files_name ='your path'# Change to the directory where your annotation system folder is located
images_name = files_name + 'fts images'  # save directory

# Custom function to convert to JPG file
def fts_to_jpg(image_fts):
    min_val = np.min(image_fts)
    max_val = np.max(image_fts)
    normalized_array = (image_fts - min_val) / (max_val - min_val)  # Normalize
    jpg_scaled_array = normalized_array * 255  # Scale to 0-255 range
    jpg_converted_array = jpg_scaled_array.astype(np.uint8)  # Convert to uint8 integer type

    # Export
    return jpg_converted_array

class ImageAnnotator:
    def __init__(self, root, img_dir, annotation_file):
        self.root = root
        self.img_dir = img_dir
        self.annotation_file = annotation_file

        # Get all fts.gz files
        self.img_files = sorted(glob.glob(os.path.join(self.img_dir, '**', '*.fts.gz'), recursive=True))

        # Load annotation file
        if os.path.exists(self.annotation_file):
            self.df = pd.read_csv(self.annotation_file)
            # Remove .fts suffix from ID column
            self.df['ID'] = self.df['ID'].apply(lambda x: x.replace('.fts', ''))
            # Delete duplicate records
            self.df.drop_duplicates(subset=['ID'], keep='last', inplace=True)
        else:
            self.df = pd.DataFrame(columns=['ID', 'filter', 'rell_number', 'date', 'hour', 'minute', 'seconds', 'FTS.GZ'])

        # Get the last annotated image
        self.img_index = self.load_progress()

        # Create GUI components
        self.create_widgets()

        # Record bounding box data
        self.current_boxes = {}
        self.active_category = None
        self.rect = None
        self.start_x = None
        self.start_y = None

        # Load the first image
        self.load_image()

    def create_widgets(self):
        # Create a Canvas to display the image
        self.canvas = tk.Canvas(self.root, cursor="cross")
        self.canvas.pack()

        # Category selection buttons
        categories = ['filter', 'rell_number', 'date', 'hour', 'minute', 'seconds']
        self.category_buttons = {}
        for cat in categories:
            btn = tk.Button(self.root, text=cat, command=lambda c=cat: self.select_category(c))
            btn.pack(side=tk.LEFT)
            self.category_buttons[cat] = btn

        # Display currently selected annotation category
        self.status_label = tk.Label(self.root, text="No annotation category selected", anchor="w")
        self.status_label.pack(fill=tk.X)

        # Image file name display
        self.image_name_label = tk.Label(self.root, text="Image Name: ", anchor="w")
        self.image_name_label.pack(fill=tk.X)

        # Save button
        self.save_button = tk.Button(self.root, text='Save', command=self.save_annotation)
        self.save_button.pack(side=tk.LEFT)

        # Image navigation buttons
        self.prev_button = tk.Button(self.root, text='Previous Image', command=self.prev_image)
        self.prev_button.pack(side=tk.LEFT)

        self.next_button = tk.Button(self.root, text='Next Image', command=self.next_image)
        self.next_button.pack(side=tk.LEFT)

        # Bind mouse events
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<B1-Motion>", self.on_mouse_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

    def load_image(self):
        """Load and display the current image"""
        fts_gz_path = self.img_files[self.img_index]
        output_dir = files_name + 'output-fts/'
        os.makedirs(output_dir, exist_ok=True)
        extracted_fits_file = os.path.join(output_dir, os.path.basename(fts_gz_path).replace('.gz', ''))

        # Decompress .fts.gz file
        with gzip.open(fts_gz_path, 'rb') as f_in:
            with open(extracted_fits_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        # Open FITS file and extract data
        hdul = fits.open(extracted_fits_file)
        data = hdul[0].data
        data = np.flipud(data)
        data = fts_to_jpg(data)

        # Convert FITS data to image
        img = Image.fromarray(data)
        img.thumbnail((800, 600))  # Resize image
        self.photo = ImageTk.PhotoImage(img)

        # Display image on Canvas
        self.canvas.config(width=self.photo.width(), height=self.photo.height())
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)

        # Display image name
        self.image_name_label.config(text=f"Image Name: {os.path.basename(fts_gz_path)}")

        # Read previous bounding boxes
        self.load_annotations_for_image()

    def load_annotations_for_image(self):
        """Load and display bounding boxes for the current image"""
        self.clear_boxes()
        img_name = os.path.basename(self.img_files[self.img_index])
        current_annotations = self.df[self.df['FTS.GZ'] == img_name]
        for _, row in current_annotations.iterrows():
            # Process bounding box coordinates, extract coordinate data from tuples
            self.create_box(
                row['filter'], row['rell_number'], row['date'],
                row['hour'], row['minute'], row['seconds']
            )

    def clear_boxes(self):
        """Clear all bounding boxes"""
        if self.rect:
            self.canvas.delete(self.rect)
        self.rect = None

    def select_category(self, category):
        """Select annotation category"""
        self.active_category = category
        # Update status bar to display current annotation category
        self.status_label.config(text=f"Current annotation category: {category}")

    def on_button_press(self, event):
        """Record initial position on mouse press"""
        if self.active_category is None:
            messagebox.showwarning("Warning", "Please select an annotation category!")
            return

        self.start_x = event.x
        self.start_y = event.y

        if self.rect:
            self.canvas.delete(self.rect)
        self.rect = self.canvas.create_rectangle(self.start_x, self.start_y, self.start_x, self.start_y, outline="red")

    def on_mouse_drag(self, event):
        """Update rectangle on mouse drag"""
        cur_x, cur_y = (event.x, event.y)
        self.canvas.coords(self.rect, self.start_x, self.start_y, cur_x, cur_y)

    def on_button_release(self, event):
        """Record final bounding box on mouse release"""
        end_x, end_y = event.x, event.y
        self.current_boxes[self.active_category] = (self.start_x, self.start_y, end_x, end_y)
        self.canvas.create_text(self.start_x, self.start_y - 10, anchor=tk.NW, text=self.active_category, fill="red")

    def prev_image(self):
        """Load previous image"""
        if self.img_index > 0:
            self.img_index -= 1
        self.load_image()

    def next_image(self):
        """Load next image"""
        if self.img_index < len(self.img_files) - 1:
            self.img_index += 1
        self.load_image()

    def save_annotation(self):
        """Save current image's annotations to CSV"""
        # Check if all categories for the current image are annotated
        missing_categories = [cat for cat in self.category_buttons if cat not in self.current_boxes]
        if missing_categories:
            messagebox.showwarning("Warning", f"Unannotated categories: {', '.join(missing_categories)}")
            return

        # Save annotation data
        img_name = os.path.basename(self.img_files[self.img_index]).replace('.gz', '')
        annotation_data = {
            'ID': img_name,
            'FTS.GZ': os.path.basename(self.img_files[self.img_index]),
            'filter': '', 'rell_number': '', 'date': '',
            'hour': '', 'minute': '', 'seconds': ''
        }

        for category, box in self.current_boxes.items():
            annotation_data[category] = box  # Save bounding box for each category
        self.df = self.df.append(annotation_data, ignore_index=True)
        self.df.to_csv(self.annotation_file, index=False)

        # Automatically move to the next image
        self.next_image()

    def load_progress(self):
        """Load progress file, return index of last processed image"""
        if os.path.exists(self.annotation_file):
            # Get the index of the last annotated image from the CSV file
            last_img_name = self.df['FTS.GZ'].iloc[-1] if not self.df.empty else None
            if last_img_name:
                # Get the index of the last annotated image in img_files, and start from the next one
                self.img_index = self.img_files.index(next(f for f in self.img_files if last_img_name in f)) + 1
                if self.img_index >= len(self.img_files):
                    self.img_index = len(self.img_files) - 1  # If out of range, set to the last image
            else:
                self.img_index = 0
        else:
            self.img_index = 0  # If no annotation file, start from the beginning
        return self.img_index

    def create_box(self, filter, rell_number, date, hour, minute, seconds):
        """Draw an annotation box and associate annotation information with it"""
        # Parse coordinate information from tuples
        filter_coords = eval(filter) if isinstance(filter, str) else filter
        rell_number_coords = eval(rell_number) if isinstance(rell_number, str) else rell_number
        date_coords = eval(date) if isinstance(date, str) else date
        hour_coords = eval(hour) if isinstance(hour, str) else hour
        minute_coords = eval(minute) if isinstance(minute, str) else minute
        seconds_coords = eval(seconds) if isinstance(seconds, str) else seconds

        # Draw annotation box using parsed coordinates
        if self.rect:
            self.canvas.delete(self.rect)
        self.rect = self.canvas.create_rectangle(
            filter_coords[0], filter_coords[1], rell_number_coords[0], rell_number_coords[1],
            outline="red"
        )

        # Display annotation information next to the box
        text = f"{filter}, {rell_number}, {date}, {hour}:{minute}:{seconds}"
        self.canvas.create_text(filter_coords[0], filter_coords[1] - 10, anchor=tk.NW, text=text, fill="red")

        # Save bounding box data to current_boxes
        self.current_boxes = {
            'filter': filter_coords,
            'rell_number': rell_number_coords,
            'date': date_coords,
            'hour': hour_coords,
            'minute': minute_coords,
            'seconds': seconds_coords
        }

# Initialize annotation tool
annotator = ImageAnnotator(root, img_dir=images_name, annotation_file = files_name + 'annotations.csv')
root.mainloop()
